# Unit 2 Assessment

In this assignment, we will focus on salary prediction. The data set for this assignment includes information on job postings. Use this data set to see if you can predict the salary of a job posting (i.e., the `Salary` column in the data set) based on the job description and other attributes of the job. This is important, because this model can make a salary recommendation as soon as a job posting is entered into a system.

## Description of Variables

The description of variables are provided in "Jobs - Data Dictionary.docx"

## Goal

Use the **jobs_alldata.csv** data set and build models to predict **salary**.

**Be careful: this is a REGRESSION task**

## Submission:

Please save and submit this Jupyter notebook file. The correctness of the code matters for your grade. **Readability and organization of your code is also important.** You may lose points for submitting unreadable/undecipherable code. Therefore, use markdown cells to create sections, and use comments where necessary.


## Important hints:

* This assignment requires you to work with a text-based column in addition to regular numeric/categorical columns. So you will have to pay attention to a few things during data processing.
* If you open the data in Excel to get familiar with it, don't save it before closing (otherwise, it messes up the values of one of the columns).
* You can do your data prep before or after the train/test split. Regardless, you should use train_test_split only once. If you find yourself using it twice, it means you are doing something wrong.
* Recommended approach: Import the data and perform the train/test split like we always do. Then, perform the following for both train and test separately (pay attention to fit_transform and transform):
    * separate the text column and process it using the text-mining tutorials (pick any approach, though I would recommend the SCIKIT tutorial). Do not try to include text processing to the pipeline - it won't work.
    * separate the numeric and categorical columns, create pipelines for them like we always do.
    * concatenate the processed text and processed numerical/categorical columns using numpy's concatenate function. See the below example for the use of this function. 

# Section 1: (8 points in total)

## Data Prep (6 points)

In [1]:
import pandas as pd
import numpy as np

In [2]:
jobs = pd.read_csv('jobs_alldata.csv')

In [3]:
jobs.head(5)

,Salary,Job Description,Location,Min_years_exp,Technical,Comm,Travel
0,67206,Civil Service Title: Regional Director Mental ...,Remote,5,2,3,0
1,88313,The New York City Comptrollerâ€™s Office Burea...,Remote,5,2,4,10-15
2,81315,With minimal supervision from the Deputy Commi...,East campus,5,3,3,5-10
3,76426,OPEN TO CURRENT BUSINESS PROMOTION COORDINATOR...,East campus,1,1,3,0
4,55675,Only candidates who are permanent in the Princ...,Southeast campus,1,1,3,5-10


In [4]:
target = jobs['Salary']

In [5]:
jobs.isna().sum()

Salary             0
Job Description    0
Location           0
Min_years_exp      0
Technical          0
Comm               0
Travel             0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(jobs, test_size=0.3)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

In [8]:
train_y = train_set[['Salary']]
test_y = test_set[['Salary']]

train_inputs = train_set.drop(['Salary'], axis=1)
test_inputs = test_set.drop(['Salary'], axis=1)


## Feature Engineering (1 points)

Create one NEW feature from existing data. You either transform a single variable, or create a new variable from existing ones. 

Grading: 
- 0.5 points for creating the new feature correctly
- 0.5 points for the justification of the new feature (i.e., why did you create this new feature)

In [9]:
#Create a flag for whether a location is remote or not

def new_rfcol(df):
    df1 = df.copy()
    df1['remote_flag'] = np.where(df1['Location'].str.lower() == 'remote', 1, 0)
    return df1[['remote_flag']]

In [10]:
train_set['Location'].value_counts()

HQ                  649
East campus         336
Remote              334
West campus         194
Southeast campus    176
Name: Location, dtype: int64

In [11]:
new_rfcol(train_set).value_counts()

remote_flag
0              1355
1               334
dtype: int64

In [12]:
train_inputs.dtypes

Job Description    object
Location           object
Min_years_exp       int64
Technical           int64
Comm                int64
Travel             object
dtype: object

In [13]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()
# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [14]:
categorical_columns.remove('Job Description')

In [15]:
categorical_columns

['Location', 'Travel']

In [16]:
numeric_columns

['Min_years_exp', 'Technical', 'Comm']

In [17]:
feat_eng_columns = ['Location']

In [18]:
train_inputs.dtypes

Job Description    object
Location           object
Min_years_exp       int64
Technical           int64
Comm                int64
Travel             object
dtype: object

In [19]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [20]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [21]:
new_rfcol = Pipeline(steps=[('rfcol', FunctionTransformer(new_rfcol)),
                               ('scaler', StandardScaler())])

In [22]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('trans', new_rfcol, feat_eng_columns)],
        remainder='drop')

In [23]:
train_x = preprocessor.fit_transform(train_inputs)
train_x

array([[ 0.55130176,  0.60451474,  1.0171042 , ...,  0.        ,
         0.        , -0.49648209],
       [ 1.1075435 ,  0.60451474, -0.13429593, ...,  0.        ,
         0.        , -0.49648209],
       [-1.11742346, -0.21822933,  1.0171042 , ...,  0.        ,
         0.        , -0.49648209],
       ...,
       [-1.11742346, -1.0409734 ,  1.0171042 , ...,  0.        ,
         1.        ,  2.01417135],
       [ 0.55130176, -1.0409734 , -0.13429593, ...,  0.        ,
         0.        , -0.49648209],
       [-1.11742346, -0.21822933, -1.28569607, ...,  0.        ,
         0.        ,  2.01417135]])

In [24]:
train_x.shape, train_set.shape, train_y.shape

((1689, 13), (1689, 7), (1689, 1))

In [25]:
test_x = preprocessor.transform(test_inputs)
test_x

array([[ 1.1075435 , -1.0409734 , -2.4370962 , ...,  0.        ,
         0.        , -0.49648209],
       [-1.11742346,  0.60451474, -0.13429593, ...,  1.        ,
         0.        , -0.49648209],
       [ 1.1075435 , -0.21822933,  1.0171042 , ...,  0.        ,
         0.        , -0.49648209],
       ...,
       [ 0.55130176, -0.21822933, -0.13429593, ...,  0.        ,
         0.        , -0.49648209],
       [ 1.1075435 , -1.0409734 , -0.13429593, ...,  1.        ,
         0.        , -0.49648209],
       [ 1.1075435 ,  2.25000288, -1.28569607, ...,  0.        ,
         0.        , -0.49648209]])

In [26]:
test_x.shape, test_set.shape

((724, 13), (724, 7))

In [27]:
train_set

,Salary,Job Description,Location,Min_years_exp,Technical,Comm,Travel
1362,69522,The Division of Preservation Finance seeks a P...,West campus,4,3,4,0
2224,52835,The NYC Department of Environmental Protection...,West campus,5,3,3,0
878,127684,About the Role: The Policy Analyst acts as a l...,East campus,1,2,4,0
1731,81091,Only candidates who are permanent in the Civil...,HQ,5,1,4,0
1025,78500,Represent the nation's largest public housing ...,East campus,1,2,3,0
...,...,...,...,...,...,...,...
1960,93873,Please read this posting carefully to make cer...,East campus,2,4,3,0
127,61597,The Agency Program Assistance (APA) Unit is re...,Remote,5,2,4,0
92,59668,SBS seeks a Project Manager to work in our Pro...,Remote,1,1,4,5-10
1073,51416,CITY OF NEW YORK OFFICE OF CHIEF MEDICAL EXAMI...,HQ,4,1,3,0


In [28]:
test_set

,Salary,Job Description,Location,Min_years_exp,Technical,Comm,Travel
2409,108122,The NYC Department of Environmental Protection...,West campus,5,1,1,0
597,63431,The Department of Design and Construction (â€œ...,West campus,1,3,3,10-15
354,63598,TASK FORCE: \t\tSocial Services UNIT:\t\t\tD...,HQ,5,2,4,0
1528,82248,The New York City Department of Environmental ...,HQ,2,3,3,0
763,72480,** THIS IS A TEMPORARY POSITION** In 2019. Ma...,East campus,1,2,4,0
...,...,...,...,...,...,...,...
2261,78450,SBS is seeking a Director for the Database Man...,East campus,1,3,3,10-15
1553,68328,In accordance to Local Law 196 established in ...,HQ,2,2,4,0
1437,102245,Please read this posting carefully to make cer...,East campus,4,2,3,0
1560,55102,A Staff Analyst (Level I) is a full-time instr...,HQ,5,1,3,10-15


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(stop_words='english')
train_x_tr = tfidf_vect.fit_transform(train_set['Job Description'])

In [30]:
test_x_tr = tfidf_vect.transform(test_set['Job Description'])

In [31]:
train_x_tr.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04631033, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [32]:
test_x_tr.toarray()

array([[0.        , 0.04584035, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.06252861, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [33]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=800, n_iter=5)

In [34]:
train_x_lsa = svd.fit_transform(train_x_tr)

In [35]:
train_x_lsa.shape

(1689, 800)

In [36]:
test_x_lsa = svd.transform(test_x_tr)

In [37]:
test_x_lsa.shape

(724, 800)

In [38]:
svd.explained_variance_ratio_.sum()

# Increasing components above 800 has a big impact on performance.  Keeping the 95% found.

0.9558007177278176

In [39]:
#Baseline
from sklearn.dummy import DummyRegressor
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(train_x, train_y)

DummyRegressor()

In [40]:
from sklearn.metrics import mean_squared_error
dummy_train_pred = dummy_regr.predict(train_x)
baseline_train_mse = mean_squared_error(train_y, dummy_train_pred)
baseline_train_rmse = np.sqrt(baseline_train_mse)
print('Baseline Train RMSE: {}' .format(baseline_train_rmse))

Baseline Train RMSE: 29841.139958866115


In [41]:
dummy_test_pred = dummy_regr.predict(test_x)
baseline_test_mse = mean_squared_error (test_y, dummy_test_pred)
baseline_test_rmse = np.sqrt(baseline_test_mse)
print('Baseline Test RMSE: {}' .format(baseline_test_rmse))

Baseline Test RMSE: 27644.169855695563


In [42]:
# Merge the train dataset
train_combined = np.concatenate((train_x,train_x_lsa), axis=1)
train_combined

array([[ 5.51301762e-01,  6.04514738e-01,  1.01710420e+00, ...,
        -1.32835859e-02, -8.35604583e-03,  7.61940031e-03],
       [ 1.10754350e+00,  6.04514738e-01, -1.34295930e-01, ...,
         5.16416540e-03,  5.13172697e-03, -3.02525505e-03],
       [-1.11742346e+00, -2.18229333e-01,  1.01710420e+00, ...,
        -7.32238611e-03,  3.09829609e-02,  1.14441603e-02],
       ...,
       [-1.11742346e+00, -1.04097340e+00,  1.01710420e+00, ...,
        -2.55369525e-03, -7.04571204e-04, -1.33057322e-02],
       [ 5.51301762e-01, -1.04097340e+00, -1.34295930e-01, ...,
        -4.18719813e-03, -1.85069923e-03, -6.36879859e-03],
       [-1.11742346e+00, -2.18229333e-01, -1.28569607e+00, ...,
        -2.95977384e-03, -1.61635994e-03,  5.88049819e-03]])

In [43]:
train_combined.shape
# shows the 1689 rows plus the 800 SVD columns and the 13 pipeline columns

(1689, 813)

In [44]:
# Merge the test dataset
test_combined = np.concatenate((test_x,test_x_lsa), axis=1)

In [45]:
test_combined.shape

(724, 813)

# Section 2: (7 points in total)

Build the following models:


## Decision Tree: (1 point)

In [46]:
from sklearn.ensemble import RandomForestRegressor
rnd_reg = RandomForestRegressor(n_estimators=200, max_depth=15,min_samples_leaf=5,n_jobs=-1) 
rnd_reg.fit(train_combined, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=15, min_samples_leaf=5, n_estimators=200,
                      n_jobs=-1)

In [47]:
#Train RMSE
train_pred = rnd_reg.predict(train_combined)
train_mse = mean_squared_error(train_y, train_pred)
train_rmse = np.sqrt(train_mse)
print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 11034.722576964012


In [48]:
#Test RMSE
test_pred = rnd_reg.predict(test_combined)
test_mse = mean_squared_error(test_y, test_pred)
test_rmse = np.sqrt(test_mse)
print('Test RMSE: {}' .format(test_rmse))

# Original: (n_estimators=300, max_depth=5,min_samples_leaf=5,n_jobs=-1)
# As I increased max_depth from 5 to 10, the overfitting worsened, but the RMSE decreased -> better model
# By then increasing min_samples_leaf = 10, the overfitting worsened, but the RMSE also increased
# Also, tried max_depth=15,min_samples_leaf=5 -> best model -- bad overfitting, but lowest RMSE of all attempts
# Started altering the n_estimators by 50, going down to 100.  300 -> 200 = lowest RMSE, trivial overfitting reduction
# This was the model with the lowest RMSE.  

Test RMSE: 17555.080393074502


## Voting regressor (2 points):

The voting regressor should have at least 3 individual models

In [49]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor 
from sklearn.svm import SVR 
from sklearn.ensemble import VotingRegressor


dtree_reg = DecisionTreeRegressor(max_depth=20)
svm_reg =SVR(kernel="rbf", C=10, epsilon=0.01, gamma='scale') 
sgd_reg = SGDRegressor(max_iter=10000, tol=1e-3)

voting_reg = VotingRegressor(
            estimators=[('dt', dtree_reg), 
                        ('svr', svm_reg), 
                        ('sgd', sgd_reg)])

voting_reg.fit(train_combined, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\ensemble\_voting.py:604: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


VotingRegressor(estimators=[('dt', DecisionTreeRegressor(max_depth=20)),
                            ('svr', SVR(C=10, epsilon=0.01)),
                            ('sgd', SGDRegressor(max_iter=10000))])

In [50]:
#Train RMSE
train_pred = voting_reg.predict(train_combined)
train_mse = mean_squared_error(train_y, train_pred)
train_rmse = np.sqrt(train_mse)
print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 13017.034789372725


In [51]:
#Test RMSE
test_pred = voting_reg.predict(test_combined)
test_mse = mean_squared_error(test_y, test_pred)
test_rmse = np.sqrt(test_mse)
print('Test RMSE: {}' .format(test_rmse))

# Original:
#dtree_reg = DecisionTreeRegressor(max_depth=20)
#svm_reg = SVR(kernel="rbf", C=10, epsilon=0.01, gamma='scale') 
#sgd_reg = SGDRegressor(max_iter=10000, tol=1e-3)
# Some overfitting, not as good as DT alone

# Increased C for SVR from 10 to 100 -> slightly worse RMSE, slight decrease in overfitting 
# Added penalty='elasticnet', l1_ratio=.25 to SGDRegressor -> worse RMSE -> slight decrease in overfitting
# Used the same param in DT -> much lower overfitting (3%), but worse RMSE (increase)
# Decreased max_iter from 10000 to 1000 for SGD -> Worse RMSE, lower overfitting

# Went back to original values 

Test RMSE: 17676.45928426047


## A Boosting model: (1 point)

Build either an Adaboost or a GradientBoost model

In [52]:
from sklearn.ensemble import GradientBoostingRegressor
gb_reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.4) 
gb_reg.fit(train_combined, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(learning_rate=0.4)

In [53]:
#Train RMSE
train_pred = gb_reg.predict(train_combined)
train_mse = mean_squared_error(train_y, train_pred)
train_rmse = np.sqrt(train_mse)
print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 3638.7863378381526


In [54]:
#Test RMSE
test_pred = gb_reg.predict(test_combined)
test_mse = mean_squared_error(test_y, test_pred)
test_rmse = np.sqrt(test_mse)
print('Test RMSE: {}' .format(test_rmse))

# Original: GradientBoostingRegressor(n_estimators=40, learning_rate=0.2) -- OVerfitting, worse than DT model RMSE (higher)
# Increased estimators from 40 to 100 and learning rate from 0.2 to 0.3 -> Overfitting, still there, much better RMSE
# Increased estimators from 100 to 120 -> Better train, worse test 
# Decreased estimated to 90 , higher RMSE (worse)

# Best model seems to be 100 estimators and a learning rate of .4



Test RMSE: 15784.563262032645


## Neural network: (1 point)

In [55]:
from sklearn.neural_network import MLPRegressor
dnn_reg = MLPRegressor(hidden_layer_sizes=(10,10,10,10), max_iter=500)
dnn_reg.fit(train_combined, train_y)

C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Sophie\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(10, 10, 10, 10), max_iter=500)

In [56]:
#Train RMSE
train_pred = dnn_reg.predict(train_combined)
train_mse = mean_squared_error(train_y, train_pred)
train_rmse = np.sqrt(train_mse)
print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 10987.435738329435


In [57]:
#Test RMSE
test_pred = dnn_reg.predict(test_combined)
test_mse = mean_squared_error(test_y, test_pred)
test_rmse = np.sqrt(test_mse)
print('Test RMSE: {}' .format(test_rmse))

# Original: MLPRegressor(hidden_layer_sizes=(10,5), max_iter=500) /No overfitting  Terrible RMSE (45667)
# Going down to one layer made performance much worse (82388)
# Went to 2 layers of 10 neurons: improved MRSE (28350) 
# Went to 2 layers of 20 neurons: improved MRSE (22994)
# Went to 3 layers of 30/20/10 neurons: improved MRSE (18585)
# 4 layers - pipe shape (all 10 neurons) No overfitting, best RMSE - Adding layers or neurons did not change much

Test RMSE: 17150.403763492122


## Grid search (2 points)

Perform either a full or randomized grid search on any model you want. There has to be at least two parameters for the search. 

In [58]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = [
    {'min_samples_leaf': np.arange(5, 20), 
     'max_depth': np.arange(5,25),
     'splitter': ['random', 'best']
    }
  ]
tree_reg = DecisionTreeRegressor()
grid_search = RandomizedSearchCV(tree_reg, param_grid, cv=6, n_iter=15,
                           scoring='neg_mean_squared_error', verbose=1,
                           return_train_score=True)
grid_search.fit(train_combined, train_y)

Fitting 6 folds for each of 15 candidates, totalling 90 fits


RandomizedSearchCV(cv=6, estimator=DecisionTreeRegressor(), n_iter=15,
                   param_distributions=[{'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24]),
                                         'min_samples_leaf': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
                                         'splitter': ['random', 'best']}],
                   return_train_score=True, scoring='neg_mean_squared_error',
                   verbose=1)

In [59]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

27515.290275421747 {'splitter': 'best', 'min_samples_leaf': 15, 'max_depth': 5}
28350.825675573622 {'splitter': 'random', 'min_samples_leaf': 6, 'max_depth': 9}
27649.73296492607 {'splitter': 'best', 'min_samples_leaf': 17, 'max_depth': 5}
27938.058275256768 {'splitter': 'random', 'min_samples_leaf': 6, 'max_depth': 17}
28510.29703153272 {'splitter': 'random', 'min_samples_leaf': 14, 'max_depth': 12}
27927.890309208316 {'splitter': 'best', 'min_samples_leaf': 15, 'max_depth': 23}
28825.134713858155 {'splitter': 'random', 'min_samples_leaf': 11, 'max_depth': 12}
27515.59518985042 {'splitter': 'best', 'min_samples_leaf': 19, 'max_depth': 22}
27450.2516184889 {'splitter': 'best', 'min_samples_leaf': 14, 'max_depth': 5}
27991.468711564266 {'splitter': 'random', 'min_samples_leaf': 6, 'max_depth': 18}
27466.31754879348 {'splitter': 'best', 'min_samples_leaf': 6, 'max_depth': 24}
27339.324470283012 {'splitter': 'best', 'min_samples_leaf': 11, 'max_depth': 13}
28522.68261713065 {'splitter': '

In [60]:
grid_search.best_params_

{'splitter': 'best', 'min_samples_leaf': 11, 'max_depth': 13}

In [61]:
grid_search.best_estimator_

DecisionTreeRegressor(max_depth=13, min_samples_leaf=11)

In [62]:
#Train RMSE
train_pred = grid_search.best_estimator_.predict(train_combined)
train_mse = mean_squared_error(train_y, train_pred)
train_rmse = np.sqrt(train_mse)
print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 15216.137468415382


In [63]:
#Test RMSE
test_pred = grid_search.best_estimator_.predict(test_combined)
test_mse = mean_squared_error(test_y, test_pred)
test_rmse = np.sqrt(test_mse)
print('Test RMSE: {}' .format(test_rmse))

# Original: param_grid = [{'min_samples_leaf': np.arange(1, 10), 'max_depth': np.arange(25,125),'splitter': ['random', 'best']}
# Bad overfitting (6000 to 22000) and worse RMSE compared to other models
# Updated to: {'min_samples_leaf': np.arange(1, 15), 'max_depth': np.arange(5,75)}  -> Worse overfitting, slightly better RMSE
# Updated to: {'min_samples_leaf': np.arange(10, 20), 'max_depth': np.arange(5,25) -> Much less overfitting, worse RMSE
# Updated to: 'min_samples_leaf': np.arange(5, 20) -> Best result for this Grid Search

Test RMSE: 24248.91060563585


# Discussion (5 points in total)


## List the train and test values of each model you built (2 points)

## Which model performs the best and why? (0.5 points) 
## How does it compare to baseline? (0.5 points)

Hint: The best model is the one that has the lowest TEST RMSE value (regardless of any of the training values). If you select your model based on TRAIN values, you will lose points.

## Is there any evidence of overfitting in the best model, why or why not? If there is, what did you do about it? (1 point)

## Is there any overfitting in the other models (besides the best model), why or why not? If there is, what did you do about it? (1 point)